In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from matplotlib.pyplot import figure
import matplotlib.dates as mdates
import datetime
import seaborn as sns

In [3]:
df_validation = pd.read_csv(r'../../data/tidy/energy_validation_april2022.csv', delimiter=',', skiprows=0, low_memory=False)

In [4]:
df_model = pd.read_csv(r'../../results/computed-fuel-rates.csv', delimiter=',', skiprows=0, low_memory=False)

In [6]:
df_validation.head()

,Unnamed: 0,timestamp,equipment_id,gallons,date
0,58932,2022-04-01 00:04:00,3213,46.9,2022-04-01
1,58933,2022-04-01 00:11:00,3303,16.0,2022-04-01
2,58934,2022-04-01 00:19:00,3322,31.7,2022-04-01
3,58935,2022-04-01 00:28:00,3312,35.2,2022-04-01
4,58936,2022-04-01 00:33:00,3307,33.4,2022-04-01


In [16]:
df_model.head()

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0,ServiceDateTime,Unnamed: 0.1,Unnamed: 0.1.1,DateKey,CalendarDate,Year,...,geometry,dist,Speed,Acceleration,time_delta,time_delta_in_seconds,Powertrain,VehicleModel,Vehicle_mass,FuelRate/Energy
0,0,1297419,1297419,2391,2022-04-29 14:01:44,2391,1273034,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.52602 42.14098),0.000000,0.000000,0.000000,0,0.0,electric,New Flyer 35 electric,26900.0,0
1,1,1299038,1299038,2392,2022-04-29 14:28:20,2392,1273037,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.65278 42.19514),8.829565,32.052117,0.005579,0 days 00:26:36,1596.0,electric,New Flyer 35 electric,26900.0,10.919433141596903
2,2,1299039,1299039,2393,2022-04-29 14:28:20,2393,1273500,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.65278 42.19514),0.000000,0.000000,0.000000,0 days 00:00:00,0.0,electric,New Flyer 35 electric,26900.0,0.0
3,3,1299250,1299250,2394,2022-04-29 14:30:55,2394,1273501,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.64553 42.20302),0.526585,19.682810,0.035274,0 days 00:02:35,155.0,electric,New Flyer 35 electric,26900.0,0.8072716996929717
4,4,1299272,1299272,2395,2022-04-29 14:31:09,2395,1273502,8788,2022-04-29 00:00:00.000,2022,...,POINT (-72.64369 42.20166),0.130185,53.874613,0.678410,0 days 00:00:14,14.0,electric,New Flyer 35 electric,26900.0,1.329511316146411


In [23]:
df_model = df_model.fillna(0)
df_model['FuelRate/Energy'] = pd.to_numeric(df_model['FuelRate/Energy'])
df_model_diesel = df_model[df_model.Powertrain != 'electric']
df_daily_fuel = df_model.groupby('Date').agg({'FuelRate/Energy':'sum'})[['FuelRate/Energy']].reset_index()

ValueError: Unable to parse string "1514      -0.000000
2340      -0.000000
2341      -0.000003
2401      -0.078061
2479      -0.028958
             ...   
1319911   -0.261029
1319922   -0.286338
1319927   -0.262999
1320030   -0.000000
1320031   -0.137818
Name: Acceleration, Length: 27848, dtype: float64" at position 5

In [25]:
pd.to_numeric(df_model['FuelRate/Energy'])

ValueError: Unable to parse string "1514      -0.000000
2340      -0.000000
2341      -0.000003
2401      -0.078061
2479      -0.028958
             ...   
1319911   -0.261029
1319922   -0.286338
1319927   -0.262999
1320030   -0.000000
1320031   -0.137818
Name: Acceleration, Length: 27848, dtype: float64" at position 5

In [ ]:
DataFrame. astype()

In [11]:
df_daily_fuel.plot(x="Date", y="FuelRate/Energy", color='teal')
plt.ylabel("Fuel consumption (gallons)", fontsize=16)
plt.xlabel("Day", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
#plt.savefig(r'../../figures/fuel-consumption-model.png', dpi=400, bbox_inches='tight')
plt.show()

TypeError: no numeric data to plot